In [ ]:
import pandas as pd
import numpy as np
import random
import os
from Celloc.common import read_data,sample_single_cells,plot_results_bulk_ST_by_spot,estimate_cell_number_RNA_reads
import matplotlib.pyplot as plt
import scanpy as sc
import sys
from Celloc import CELLOC
import anndata as ad

# Load data

Read simulated mouse cerebellum datasets

In [ ]:
scRNA_path="./data/Cerebellum/scRNA/scRNA_Noise05.csv"
cell_type_path="./data/Cerebellum/scRNA/cell_labels.csv"
st_path="./data/Cerebellum/5/ST_expression.csv"
coordinates_path="./data/Cerebellum/5/Coordinates.csv"
output_folder="./Celloc_output/Cerebellum/"
mean_cell_numbers=5

Read simulated mouse hippocampus datasets

In [ ]:
# scRNA_path="./data/Hippocampus/scRNA/scRNA_Noise05.csv"
# cell_type_path="./data/Hippocampus/scRNA/cell_labels.csv"
# st_path="./data/Hippocampus/5/ST_expression.csv"
# coordinates_path="./data/Hippocampus/5/Coordinates.csv"
# mean_cell_numbers=5

Read the HER2+ breast cancer scRNA-seq data and 10X Visium ST samples1/ST samples2

In [ ]:
# scRNA_path="./data/breast_cancer1/brca_scRNA_GEP.txt"
# cell_type_path="./data/breast_cancer1/brca_scRNA_celllabels.txt"

## using 10X Visium ST samples1
# st_path="./data/breast_cancer1/spatial.h5ad"

## using 10X Visium ST samples2
# st_path="./data/breast_cancer2/spatial.h5ad"

# coordinates_path=None

Read the DCIS1/DCIS2 datasets

In [ ]:
## reading DCIS1
# scRNA_path="./data/DCIS1/sc_RNA_DCIS1.h5ad"
# cell_type_path=None
# st_path="./data/DCIS1/spatial_DCIS1.h5ad"
# coordinates_path=None

## reading DCIS2
# scRNA_path="./data/DCIS2/sc_RNA_DCIS2.h5ad"
# cell_type_path=None
# st_path="./data/DCIS2/spatial_DCIS2.h5ad"
# coordinates_path=None
# mean_cell_numbers=5

Read the myocardial infarction scRNA-seq data and ST data

In [ ]:
# scRNA_path="./data/heart_data/scfile.txt"
# cell_type_path="./data/heart_data/sclab.txt"
# st_path="./data/heart_data/spatial.h5ad"
# coordinates_path=None
# celltype_col='ct'

In [ ]:
celltype_col='CellType'

# Read the data from the specified paths
# sc_adata: single-cell RNA-seq data
# spatial_adata: spatial transcriptomics data
# cell_type_number_eachspot_data: estimated number of each cell type in each spot
# coordinates_data: coordinates of each spot
sc_adata, spatial_adata, coordinates_data =\
        read_data(scRNA_path, cell_type_path, st_path, coordinates_path,celltype_col)

# Extract the cell type information from the single-cell data
# Rename the index and the 'CellType' column for clarity
cell_type=pd.DataFrame(sc_adata.obs['CellType'])
cell_type.index = [str(idx)[5:] for idx in cell_type.index]
cell_type['CellType'] = [str(idx)[5:] for idx in cell_type['CellType']]

# estimate the number of cells in each spot
cell_number_to_node_assignment=estimate_cell_number_RNA_reads(spatial_adata.to_df().T, int(mean_cell_numbers))

In [ ]:
# Filter out genes that are expressed in less than 1 cell in the single-cell data
sc.pp.filter_genes(sc_adata, min_cells=1)

# Filter out genes that are expressed in less than 1 cell in the spatial data
sc.pp.filter_genes(spatial_adata, min_cells=1)

# Find the intersection of the gene lists from the single-cell and spatial data
intersect_genes = list(set(sc_adata.var_names) & set(spatial_adata.var_names))

# Subset the single-cell and spatial data to only include the intersecting genes
sc_adata = sc_adata[:,intersect_genes]
spatial_adata = spatial_adata[:,intersect_genes]

In [ ]:
# Normalize the total counts in each cell in the sampled single-cell data to 1e6
sc.pp.normalize_total(sc_adata,target_sum=10**6)

# Log-transform the data in the sampled single-cell data
sc.pp.log1p(sc_adata)

# Normalize the total counts in each cell in the spatial data to 1e6
sc.pp.normalize_total(spatial_adata,target_sum=10**6)

# Log-transform the data in the spatial data
sc.pp.log1p(spatial_adata)

# Perform PCA on the sampled single-cell data, keeping the first 50 principal components
sc.tl.pca(sc_adata,n_comps=50)

print(sc_adata)
print(spatial_adata)

In [ ]:
# Use the CELLOC library's paired_align function to align the sampled single-cell data (sc_adata_sampled) and the spatial data (spatial_adata)
# The initial guess for the number of cells in each spot (b_init) is provided by cell_number_to_node_assignment
# The task is set to "mapping", which means the function will try to map single cell to a spot
pi_sc_sp = CELLOC.paired_align(sc_adata, spatial_adata, b_init=cell_number_to_node_assignment, task="mapping")

# Convert the output of the paired_align function to a pandas DataFrame
# The index is the cell names from the sampled single-cell data, and the columns are the spot names from the spatial data
sc_sp_map_df = pd.DataFrame(pi_sc_sp, index=sc_adata.obs_names, columns=spatial_adata.obs_names)

In [ ]:
# Remove the first 5 characters from the column and index names of the DataFrame
sc_sp_map_df.columns = [str(col)[5:] for col in sc_sp_map_df.columns]
sc_sp_map_df.index = [str(idx)[5:] for idx in sc_sp_map_df.index]

# Initialize an empty dictionary to store the results
result_dict = {}

# For each column in the DataFrame, find the cell names with the highest values
# The number of cell names to find is determined by cell_number_to_node_assignment
# Store the results in the result_dict dictionary
for i, column in enumerate(sc_sp_map_df.columns.values):
    result_dict[column] = sc_sp_map_df[column].nlargest(cell_number_to_node_assignment[i]).index

# Convert the result_dict dictionary to a DataFrame
# The 'Predict' column contains the spot names, and the 'Values' column contains the cell names
# The 'Values' column is exploded so that each cell name has its own row
max_column_names = pd.DataFrame({'Predict': list(result_dict.keys()), 'Values': [list(index_obj) for index_obj in result_dict.values()]}).explode('Values')

# Convert the cell names in the 'Values' column to strings
max_column_names['Values'] = max_column_names['Values'].astype(str)

# Set the 'Values' column as the index of the DataFrame
max_column_names = max_column_names.set_index('Values')

# Add a new column 'CellType' to the DataFrame and initialize it with 'type'
max_column_names=max_column_names.assign(CellType='type')

# Remove rows where the index is "nan"
max_column_names = max_column_names[max_column_names.index!="nan"]

# For each row in the DataFrame, replace 'type' in the 'CellType' column with the actual cell type from the cell_type DataFrame
for i,cell_id in enumerate(max_column_names.index.values):
    max_column_names.iloc[i,-1]=cell_type.loc[cell_id][0]

# Reindex the coordinates_data DataFrame based on the 'Predict' column of the max_column_names DataFrame
# This gives the predicted coordinates for each cell
predict_coordinates=coordinates_data.reindex(max_column_names['Predict'])

# Add the predicted x and y coordinates to the max_column_names DataFrame
max_column_names['predict_x']=predict_coordinates['X'].values
max_column_names['predict_y']=predict_coordinates['Y'].values

# Print the DataFrame
print(max_column_names)

In [ ]:
assigned_locations_path = os.path.join(output_folder+"mapping_results.csv")
max_column_names.to_csv(assigned_locations_path)

Visualize the mapping results of simulated mouse cerebellum and hippocampus (Supplementary Figure 3, 4)

In [ ]:
plot_results_bulk_ST_by_spot(assigned_locations=max_column_names, coordinates_data=coordinates_data, dir_out=output_folder, output_prefix="Celloc_mapping")